In [ ]:
#download libs
!pip install langchain
!pip install PyPDF2
!pip install streamlit
!pip install sentence_transformers
!pip install openai
!pip install tiktoken
!pip install faiss-cpu

In [14]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 52.2 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
HF_KEY = userdata.get('HF_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [17]:
from google.colab import userdata
HF_KEY = userdata.get('HF_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

#importação libs
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

#essa função tem o papel de extrair os textos dos PDFs
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

#essa função tem o papel de dividir o texto em pedaços menores, a fim de otimizar a busca
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

#essa função tem o papel de criar o vetorstore, que é a base de dados que será usada para a busca das respostas, usando a OpenAI
def get_vectorstore(text_chunks):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

#essa função tem o papel de criar a cadeia de conversação, que é a base para a conversa com o PDF
def get_conversation_chain(vectorstore):
    llm = ChatOpenAI(temperature=0.0) #0.0 para respostas mais precisas
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

#essa função tem o papel de tratar a entrada do usuário e retornar a resposta do PDF
def handle_userinput(user_question, conversation):
    response = conversation({'question': user_question})
    chat_history = response['chat_history']

    # Imprime apenas a última pergunta e resposta
    print(f"Usuário: {chat_history[-2].content}")
    print(f"Modelo 🤖: {chat_history[-1].content}\n")

    # Limpa a memória da conversa
    conversation.memory.clear()

def main():

    pdf_docs = input("Insira o caminho para o seu PDF: ").split(',')
    raw_text = get_pdf_text(pdf_docs)
    text_chunks = get_text_chunks(raw_text)
    vectorstore = get_vectorstore(text_chunks)
    conversation = get_conversation_chain(vectorstore)

    while True:
        user_question = input("\nFique à vontade para perguntar qualquer coisa para o PDF: ")
        handle_userinput(user_question, conversation)

if __name__ == '__main__':
    main()

Insira o caminho para o seu PDF: /content/edital n 02 concessaopdf.pdf

Fique à vontade para perguntar qualquer coisa para o PDF: do que se trata o pdf?
Usuário: do que se trata o pdf?
Modelo 🤖: O PDF trata de um edital de licitação para a concessão de uso de três espaços físicos destinados à exploração comercial. O documento contém informações sobre o julgamento da licitação, as disposições finais, as penalidades para fraude e corrupção, entre outros detalhes relacionados ao processo licitatório.



KeyboardInterrupt: Interrupted by user